# Tecnica Árvore de Decisão para Regressão

Download do Dataset
Objetivo: Obter o dataset necessário para análise e modelagem. Este é o passo inicial que permite termos acesso aos dados.

In [48]:
import kagglehub

# Fazer o download da versão mais recente do dataset
path = kagglehub.dataset_download("iliassekkaf/computerparts")

# Verificar o caminho dos arquivos do dataset
print("Path to dataset files:", path)


Path to dataset files: C:\Users\jonas\.cache\kagglehub\datasets\iliassekkaf\computerparts\versions\7


Listar os Arquivos no Diretório.
Objetivo: Verificar o conteúdo do diretório especificado pela variável path para garantir que os arquivos necessários estão presentes antes de prosseguir com outras operações. Isso é útil para visualizar os arquivos disponíveis no diretório e confirmar que o download foi bem-sucedido.

In [49]:
import os

# Listar todos os arquivos no diretório especificado
files = os.listdir(path)
print("Arquivos no diretório:")
for file in files:
    print(file)


Arquivos no diretório:
All_GPUs.csv
Intel_CPUs.csv


Carregamento dos Dados

Objetivo: Carregar os dados em um formato manipulável usando a biblioteca pandas para análise e pré-processamento. Isso nos ajuda a visualizar e entender a estrutura dos dados.



In [50]:
import pandas as pd

# Definir o nome do arquivo CSV com base na listagem anterior
dataset_file_name = "Intel_CPUs.csv"
dataset_path = os.path.join(path, dataset_file_name)

# Carregar os dados em um DataFrame do pandas
df = pd.read_csv(dataset_path)

# Exibir as primeiras linhas do dataset para verificação
print("Primeiras linhas do dataset:")
print(df.head())


Primeiras linhas do dataset:
                          Product_Collection Vertical_Segment  \
0  7th Generation Intel® Core™ i7 Processors           Mobile   
1  8th Generation Intel® Core™ i5 Processors           Mobile   
2  8th Generation Intel® Core™ i7 Processors           Mobile   
3           Intel® Core™ X-series Processors          Desktop   
4  7th Generation Intel® Core™ i5 Processors           Mobile   

  Processor_Number       Status Launch_Date Lithography  \
0          i7-7Y75     Launched       Q3'16       14 nm   
1         i5-8250U     Launched       Q3'17       14 nm   
2         i7-8550U     Launched       Q3'17       14 nm   
3          i7-3820  End of Life       Q1'12       32 nm   
4          i5-7Y57     Launched       Q1'17       14 nm   

  Recommended_Customer_Price  nb_of_Cores  nb_of_Threads  \
0                   $393.00             2            4.0   
1                   $297.00             4            8.0   
2                   $409.00             4    

Pré-processamento dos Dados

Objetivo: Converter valores de frequência de string (por exemplo, 3.40 GHz) para float, permitindo que o modelo de aprendizado de máquina utilize essas informações corretamente. Além disso, remover linhas com valores ausentes na coluna Processor_Base_Frequency garante que o conjunto de dados esteja limpo e pronto para análise.

In [57]:
# Função para converter frequências de string para float
def convert_ghz_to_float(frequency):
    if isinstance(frequency, str):
        try:
            return float(frequency.replace(' GHz', '').replace(',', '.'))
        except ValueError:
            return None  # Retornar None se a conversão falhar
    return frequency

# Aplicar a função na coluna 'Processor_Base_Frequency'
df['Processor_Base_Frequency'] = df['Processor_Base_Frequency'].apply(convert_ghz_to_float)

# Remover linhas com valores ausentes na coluna 'Processor_Base_Frequency'
df = df.dropna(subset=['Processor_Base_Frequency'])

# Exibir as primeiras linhas após o pré-processamento para verificação
print("Primeiras linhas após o pré-processamento:")
print(df.head())


Primeiras linhas após o pré-processamento:
                          Product_Collection Vertical_Segment  \
0  7th Generation Intel® Core™ i7 Processors           Mobile   
1  8th Generation Intel® Core™ i5 Processors           Mobile   
2  8th Generation Intel® Core™ i7 Processors           Mobile   
3           Intel® Core™ X-series Processors          Desktop   
4  7th Generation Intel® Core™ i5 Processors           Mobile   

  Processor_Number       Status Launch_Date Lithography  \
0          i7-7Y75     Launched       Q3'16       14 nm   
1         i5-8250U     Launched       Q3'17       14 nm   
2         i7-8550U     Launched       Q3'17       14 nm   
3          i7-3820  End of Life       Q1'12       32 nm   
4          i5-7Y57     Launched       Q1'17       14 nm   

   Recommended_Customer_Price  nb_of_Cores  nb_of_Threads  \
0                       393.0            2            4.0   
1                       297.0            4            8.0   
2                       409.

Divisão dos Dados em Treinamento e Teste

Objetivo: Separar os dados em dois conjuntos: um para treinar o modelo (`X_train` e `y_train`) e outro para avaliar seu desempenho (`X_test` e `y_test`). Isso garante que o modelo possa ser validado em dados que ele não viu durante o treinamento, proporcionando uma avaliação justa e evitando overfitting.

In [62]:
from sklearn.model_selection import train_test_split

# Definir as features e o target
X = df.drop(columns=['Recommended_Customer_Price'])
y = df['Recommended_Customer_Price']

# Dividir os dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Mostrar a quantidade de dados em cada conjunto
print("Quantidade de dados no conjunto de treinamento:", len(X_train))
print("Quantidade de dados no conjunto de teste:", len(X_test))

# Mostrar as primeiras linhas dos conjuntos de treinamento e teste
print("\nPrimeiras linhas do conjunto de treinamento (X_train):")
print(X_train.head())

print("\nPrimeiras linhas do conjunto de teste (X_test):")
print(X_test.head())


Quantidade de dados no conjunto de treinamento: 1027
Quantidade de dados no conjunto de teste: 257

Primeiras linhas do conjunto de treinamento (X_train):
                             Product_Collection Vertical_Segment  \
1384  4th Generation Intel® Core™ i5 Processors          Desktop   
1546             Legacy Intel® Core™ Processors          Desktop   
2062           Legacy Intel® Celeron® Processor           Mobile   
2268  6th Generation Intel® Core™ i3 Processors          Desktop   
440            Intel® Core™ X-series Processors           Mobile   

     Processor_Number                      Status Launch_Date Lithography  \
1384         i5-4590S                    Launched       Q2'14       22 nm   
1546            E7400  End of Interactive Support       Q1'08       45 nm   
2062            P4500  End of Interactive Support       Q1'10       32 nm   
2268          i3-6100                    Launched       Q3'15       14 nm   
440         i7-3920XM                    Launched  

Pré-processamento Adicional

Objetivo: Aplicar transformações específicas nas colunas numéricas e categóricas para melhorar o desempenho do modelo.

OneHotEncoder para colunas categóricas: Converte variáveis categóricas em uma forma binária que o modelo pode entender, criando novas colunas para cada categoria.

StandardScaler para colunas numéricas: Normaliza os dados numéricos para que todos tenham a mesma escala, melhorando a convergência do modelo.

ColumnTransformer: Aplica essas transformações de forma consistente e eficiente aos dados de treinamento e teste.

In [59]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Identificar colunas numéricas e categóricas
numeric_features = ['nb_of_Cores', 'nb_of_Threads', 'Processor_Base_Frequency']
categorical_features = ['Product_Collection', 'Vertical_Segment', 'Status', 'Lithography']

# Definir o pré-processamento para colunas numéricas e categóricas
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Aplicar pré-processamento às colunas numéricas e categóricas
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Aplicar o pré-processamento aos dados de treinamento e teste
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print("Dados após pré-processamento:")
print(X_train_processed)


Dados após pré-processamento:
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 7189 stored elements and shape (1027, 83)>
  Coords	Values
  (0, 0)	-0.20524999807868408
  (0, 1)	-0.579966001067254
  (0, 2)	0.7651732725035156
  (0, 4)	1.0
  (0, 68)	1.0
  (0, 74)	1.0
  (0, 77)	1.0
  (1, 0)	-0.5150100980446041
  (1, 1)	nan
  (1, 2)	0.46785839570690896
  (1, 65)	1.0
  (1, 68)	1.0
  (1, 72)	1.0
  (1, 79)	1.0
  (2, 0)	-0.5150100980446041
  (2, 1)	-0.7929959562391212
  (2, 2)	-0.9295215252371409
  (2, 64)	1.0
  (2, 70)	1.0
  (2, 72)	1.0
  (2, 78)	1.0
  (3, 0)	-0.5150100980446041
  (3, 1)	-0.579966001067254
  (3, 2)	1.8057753412916384
  (3, 10)	1.0
  :	:
  (1023, 62)	1.0
  (1023, 71)	1.0
  (1023, 74)	1.0
  (1023, 76)	1.0
  (1024, 0)	1.6533106017168355
  (1024, 1)	2.4024533713388867
  (1024, 2)	-0.7214011114795166
  (1024, 62)	1.0
  (1024, 71)	1.0
  (1024, 74)	1.0
  (1024, 76)	1.0
  (1025, 0)	-0.20524999807868408
  (1025, 1)	nan
  (1025, 2)	0.2597379819492849
  (1025, 65)	1.0
  (102

Treinamento do Modelo de Árvore de Decisão

Objetivo: Inicializar e treinar um modelo de árvore de decisão (`DecisionTreeRegressor`) usando os dados de treinamento processados. Isso envolve verificar se os dados estão no formato correto (array numpy) e converter se necessário, garantindo que o modelo possa aprender os padrões nos dados e fazer previsões futuras. O `random_state` é definido para garantir a reprodutibilidade dos resultados.


In [60]:
from sklearn.tree import DecisionTreeRegressor
from scipy.sparse import issparse

# Verificar o formato dos dados processados e converter se necessário
if issparse(X_train_processed):
    X_train_processed = X_train_processed.toarray()
if issparse(X_test_processed):
    X_test_processed = X_test_processed.toarray()

# Inicializar o modelo
model = DecisionTreeRegressor(random_state=42)

# Treinar o modelo
model.fit(X_train_processed, y_train)

print("Modelo treinado com sucesso.")


Modelo treinado com sucesso.


Avaliação do Modelo

Objetivo: Avaliar o desempenho do modelo usando o conjunto de teste. Isso envolve:
- Fazer previsões (`y_pred`) no conjunto de teste (`X_test_processed`).
- Calcular o **Mean Squared Error (MSE)** para medir a média dos quadrados dos erros entre os valores previstos e os valores reais.
- Calcular o **R² Score** para medir a proporção da variância na variável dependente que é explicada pelo modelo.
- Exibir as métricas de desempenho para verificar a precisão e a eficácia do modelo.

In [61]:
from sklearn.metrics import mean_squared_error, r2_score

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test_processed)

# Calcular MSE e R²
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R² Score: {r2}")


Mean Squared Error: 876970.4128296585
R² Score: 0.579372132757934


Resultados do Modelo Resumidos

- Mean Squared Error (MSE): 876970.41
  - Resumo: Indica a média dos quadrados dos erros entre as previsões e os valores reais. Um valor mais baixo é melhor.

- R² Score: 0.579
  - Resumo: Aproximadamente 57.9% da variabilidade nos preços recomendados é explicada pelo modelo. Bom desempenho, mas ainda há margem para melhorias.